In [2]:
import pandas as pd

raw_path = "../../Formato_dataset_productos.csv"
df = pd.read_csv(raw_path, low_memory=False)

# Ver primeras filas y columnas
df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124514 entries, 0 to 124513
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   FECHA_CORTE    124514 non-null  int64  
 1   FECHA_MUESTRA  124514 non-null  int64  
 2   DEPARTAMENTO   124514 non-null  object 
 3   PROVINCIA      124514 non-null  object 
 4   DISTRITO       124514 non-null  object 
 5   UBIGEO         124514 non-null  int64  
 6   ANO            124514 non-null  int64  
 7   MES            124514 non-null  int64  
 8   COD_CULTIVO    124514 non-null  int64  
 9   CULTIVO        124514 non-null  object 
 10  SIEMBRA        124514 non-null  float64
 11  COSECHA        116335 non-null  float64
 12  PRODUCCION     124514 non-null  float64
 13  VERDE_ACTUAL   124512 non-null  float64
 14  PRECIO_CHACRA  124514 non-null  float64
dtypes: float64(5), int64(6), object(4)
memory usage: 14.2+ MB


,FECHA_CORTE,FECHA_MUESTRA,UBIGEO,ANO,MES,COD_CULTIVO,SIEMBRA,COSECHA,PRODUCCION,VERDE_ACTUAL,PRECIO_CHACRA
count,124514.0,1.245140e+05,124514.000000,124514.000000,124514.000000,1.245140e+05,124514.000000,116335.000000,124514.000000,124512.000000,124514.000000
mean,20250527.0,2.019984e+07,200338.110662,2019.861004,201992.693239,1.366054e+10,9.582755,17.217866,151.504984,152.651492,0.473059
std,0.0,2.715068e+04,190.429778,2.715068,271.255753,8.366462e+08,97.202457,205.738878,1716.729435,762.823342,1.255347
min,20250527.0,2.015123e+07,200101.000000,2015.000000,201508.000000,1.201001e+10,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20250527.0,2.018123e+07,200203.000000,2018.000000,201801.000000,1.301023e+10,0.000000,0.000000,0.000000,4.000000,0.000000
50%,20250527.0,2.020123e+07,200305.000000,2020.000000,202006.000000,1.401007e+10,0.000000,0.000000,0.000000,14.000000,0.000000
75%,20250527.0,2.022123e+07,200409.000000,2022.000000,202209.000000,1.404014e+10,0.000000,0.000000,4.000000,60.000000,0.460000
max,20250527.0,2.024123e+07,200806.000000,2024.000000,202412.000000,1.502002e+10,5500.000000,20512.000000,147000.000000,21444.750000,27.000000


In [7]:
# Formatear fecha y extraer año y mes
df["FECHA_MUESTRA"] = pd.to_datetime(df["FECHA_MUESTRA"].astype(str), format="%Y%m%d", errors="coerce")
df["MES_STR"] = df["MES"].astype(str)
df["ANO"] = df["MES_STR"].str[:4].astype(int)
df["MES_NUM"] = df["MES_STR"].str[4:6].astype(int)
df["FECHA_YYYYMM"] = pd.to_datetime(df["ANO"].astype(str) + df["MES_NUM"].astype(str).str.zfill(2), format="%Y%m")

# Formatear UBIGEO a 6 dígitos con ceros a la izquierda
df["UBIGEO"] = df["UBIGEO"].astype(str).str.zfill(6)

In [11]:
# Verificar valores numericos
for col in ["SIEMBRA","COSECHA","PRODUCCION","VERDE_ACTUAL","PRECIO_CHACRA"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Convertir produccion de toneladas a kilogramos
df["PRODUCCION_KG"] = df["PRODUCCION"] * 1000


In [9]:
# Rendimiento en toneladas por hectárea
df["REND_THA"] = df.apply(
    lambda r: (r["PRODUCCION"] / r["COSECHA"]) if r["COSECHA"] and r["COSECHA"]>0 else None, 
    axis=1
)

df[["PRODUCCION","COSECHA","REND_THA"]].head()

,PRODUCCION,COSECHA,REND_THA
0,10.0,0.0,NaN
1,0.0,0.0,NaN
2,0.0,0.0,NaN
3,76.0,2.0,38.0
4,0.0,0.0,NaN
